# Packages

In [1]:
import pandas as pd
import itertools
import networkx as nx
from itertools import combinations
import spacy
from nltk.stem import SnowballStemmer
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import warnings
import scipy.sparse.linalg
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# From modules

In [2]:
from graph_func import create_graph_from_dataframe, analyze_graph, charac_graph, charac_node, charac_edge
from text_func import wordopt, process_text, sentence_processed, delete_rows_by_values, preprocess_natural_language, preprocessed_data, calculate_sentiment

# Instalation

In [3]:
warnings.filterwarnings("ignore")

In [4]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\izaol\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\izaol\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Columns

In [6]:
col_name = ["json ID", "label", "statement", "subject", "speaker", "job title", "state", "party",
                         "barely true", "false", "half true", "mostly true", "pants on fire", "los", "justification"]

In [7]:
not_need_col = ["subject", "speaker", "job title", "state", "party",
                         "barely true", "false", "half true", "mostly true", "pants on fire", "los", "justification"]

# Extra stopwords

In [8]:
extra_stopwords = ['say', 'percent', 's', 'new', 'u', 'time', 'bill', 'cut', 'said', 'pay', 'wisconsin', 'school', 'plan', 'day',
 'florida', 'governor', 'average', 'house', 'county', 'debt', 'public', 'half', 'spending', 'number', 'world',
 'today', 'increase', 'want', 'family', 'highest', 'month', 'program', 'gov', 'campaign', 'dont', 't', 'romney',
 'history', 'going', 'member', 'john', 'system', 'actually', 'took', 'employee', 'like', 'wage', 'spent', 'paid',
 'mitt', 'weve', 'worker', 'funding', 'fund', 'walker', 'rick', 'court', 'raise', 'act', 'proposed', 'passed',
 'rhode', 'got', 'jersey', 'higher', 'high', 'work', 'sen', 'come', 'record', 'single', 'use', 'defict', 'largest',
 'second', 'change', 'term', 'lost', 'district', 'home', 'union', 'level', 'supported', 'george', 'milwaukee', 'lowest',
 'ago', 'working', 'know', 'past', 'minimum', 'citizen', 'small', 'deal', 'stimulus', 'spend', 'went', 'coverge', 'york',
 'raised', 'thats', 'think', 'called', 'week', 'check', 'white', 'didnt', 'teacher', 'came', 'away']

# Load dataset

In [9]:
train_df = pd.read_csv('fake_news_dataset/train2.tsv', sep='\t', header=None, names = col_name)

In [10]:
valid_df = pd.read_csv('fake_news_dataset/val2.tsv', sep='\t', header=None, names = col_name)

In [11]:
test_df = pd.read_csv('fake_news_dataset/test2.tsv', sep='\t', header=None, names = col_name)

# Preprocess text

In [12]:
train_df.drop(not_need_col, axis=1)

,json ID,label,statement
0.0,2635.json,false,Says the Annies List political group supports ...
1.0,10540.json,half-true,When did the decline of coal start? It started...
2.0,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo..."
3.0,1123.json,false,Health care reform legislation is likely to ma...
4.0,9028.json,half-true,The economic turnaround started at the end of ...
...,...,...,...
10264.0,5473.json,mostly-true,There are a larger number of shark attacks in ...
10265.0,3408.json,mostly-true,Democrats have now become the party of the [At...
10266.0,3959.json,half-true,Says an alternative to Social Security that op...
10267.0,2253.json,false,On lifting the U.S. Cuban embargo and allowing...


In [13]:
valid_df.drop(not_need_col, axis=1)

,json ID,label,statement
0,12134.json,barely-true,We have less Americans working now than in the...
1,238.json,pants-fire,"When Obama was sworn into office, he DID NOT u..."
2,7891.json,false,Says Having organizations parading as being so...
3,8169.json,half-true,Says nearly half of Oregons children are poor.
4,929.json,half-true,On attacks by Republicans that various program...
...,...,...,...
1279,3419.json,half-true,"For the first time in more than a decade, impo..."
1280,12548.json,mostly-true,Says Donald Trump has bankrupted his companies...
1281,401.json,true,"John McCain and George Bush have ""absolutely n..."
1282,1055.json,false,A new poll shows 62 percent support the presid...


In [14]:
test_df.drop(not_need_col, axis=1)

,json ID,label,statement
0,11972.json,true,Building a wall on the U.S.-Mexico border will...
1,11685.json,false,Wisconsin is on pace to double the number of l...
2,11096.json,false,Says John McCain has done nothing to help the ...
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...
4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...
...,...,...,...
1278,7334.json,half-true,Says his budget provides the highest state fun...
1279,9788.json,barely-true,Ive been here almost every day.
1280,10710.json,barely-true,"In the early 1980s, Sen. Edward Kennedy secret..."
1281,3186.json,barely-true,Says an EPA permit languished under Strickland...


In [15]:
train_df = preprocessed_data(train_df)
valid_df = preprocessed_data(valid_df)
test_df = preprocessed_data(test_df)

# Preprocess natural language

In [16]:
train_df = preprocess_natural_language(train_df)
valid_df = preprocess_natural_language(valid_df)
test_df = preprocess_natural_language(test_df)

Says the Annies List political group supports third-trimester abortions on demand.
Hillary Clinton agrees with John McCain "by voting to give George Bush the benefit of the doubt on Iran."
Health care reform legislation is likely to mandate free sex change surgeries.
The Chicago Bears have had more starting quarterbacks in the last 10 years than the total number of tenured (UW) faculty fired during the last two decades.
Says GOP primary opponents Glenn Grothman and Joe Leibham cast a compromise vote that cost $788 million in higher electricity costs.
For the first time in history, the share of the national popular vote margin is smaller than the Latino vote margin.
When Mitt Romney was governor of Massachusetts, we didnt just slow the rate of growth of our government, we actually cut it.
The economy bled $24 billion due to the government shutdown.
McCain opposed a requirement that the government buy American-made motorcycles. And he said all buy-American provisions were quote 'disgrace

# Graph

### All data

In [17]:
G = create_graph_from_dataframe(train_df, 'preprocess_tokens')

In [18]:
charac_graph(G)

Rząd - liczba wierzchołków:  7325
Rozmiar - liczba krawędzi:  213813
Gęstość grafu:  0.00797091426941767



KeyboardInterrupt



In [93]:
charac_edge(G)

KeyboardInterrupt: 

In [ ]:
charac_node(G)

### True news

In [ ]:
G_true = create_graph_from_dataframe(train_df[train_df['label'] == 'true'], 'preprocess_tokens')

In [ ]:
charac_graph(G_true)

In [ ]:
charac_edge(G_true)

In [ ]:
charac_node(G_true)

### Fake news

In [ ]:
G_false = create_graph_from_dataframe(train_df[train_df['label'] == 'false'], 'preprocess_tokens')

In [ ]:
charac_graph(G_false)

In [ ]:
charac_edge(G_false)

In [ ]:
charac_node(G_false)

# Fake news detector

In [22]:
from sklearn.utils import resample

In [20]:
train_df['sentence_processed'] = train_df['preprocess_tokens'].apply(sentence_processed)
valid_df['sentence_processed'] = valid_df['preprocess_tokens'].apply(sentence_processed)
test_df['sentence_processed'] = test_df['preprocess_tokens'].apply(sentence_processed)

In [60]:
tfidf_vectorizer=TfidfVectorizer()

In [61]:
tfidf_train = tfidf_vectorizer.fit_transform(train_df['sentence_processed']) 
tfidf_test = tfidf_vectorizer.transform(test_df['sentence_processed']) 

In [62]:
tfidf_test

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5928 stored elements and shape (698, 7305)>

In [63]:
tfidf_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 50984 stored elements and shape (5633, 7305)>

In [41]:
y_train = train_df['label']
y_test = test_df['label']

# Classifier

### logistic regression

In [73]:
def train_models(tfidf_train, y_train):
    
    # logistic regression
    
    lr_params = {
    'class_weight': ['balanced'],
    'penalty': ['l1', 'l2', 'elasticnet', None],
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'saga']
    }
    
    lr = GridSearchCV(LogisticRegression(max_iter=1000), lr_params, cv=5, scoring='accuracy', n_jobs=-1)
    lr.fit(tfidf_train, y_train)
    
    # svm
    
    svm_params = {
    'class_weight': ['balanced'],
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']
    }
    
    svm = GridSearchCV(SVC(), svm_params, cv=5, scoring='accuracy', n_jobs=-1)
    svm.fit(tfidf_train, y_train)
    
    # knn
    
    knn_params = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'p': [1, 2] 
    }
    knn = GridSearchCV(KNeighborsClassifier(), knn_params, cv=5, scoring='accuracy', n_jobs=-1)
    knn.fit(tfidf_train, y_train)
    
    # models
    
    models = {
    "Logistic Regression": lr,
    "SVM": svm,
    "KNN": knn
    }
    
    return models

In [75]:
def model_result(models):
    
    for model_name, model in models.items():
        print(f"\nBest parameters for {model_name}: {model.best_params_}")
        print(f"Best cross-validation score for {model_name}: {model.best_score_:.4f}")

    for model_name, model in models.items():
        best_model = model.best_estimator_
        y_pred = best_model.predict(tfidf_test)
        print(f"\n{model_name} - Test Set Performance:")
        print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
        print(classification_report(y_test, y_pred))

In [68]:
models = train_models(tfidf_train, y_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5928 stored elements and shape (698, 7305)>

In [69]:
model_result(models)


Best parameters for Logistic Regression: {'C': 1, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}
Best cross-validation score for Logistic Regression: 0.6174

Best parameters for SVM: {'C': 1, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'poly'}
Best cross-validation score for SVM: 0.6631

Best parameters for KNN: {'n_neighbors': 7, 'p': 2, 'weights': 'distance'}
Best cross-validation score for KNN: 0.6233

Logistic Regression - Test Set Performance:
Accuracy: 0.6232
              precision    recall  f1-score   support

       false       0.47      0.53      0.50       249
        true       0.72      0.68      0.70       449

    accuracy                           0.62       698
   macro avg       0.60      0.60      0.60       698
weighted avg       0.63      0.62      0.63       698

SVM - Test Set Performance:
Accuracy: 0.6533
              precision    recall  f1-score   support

       false       0.61      0.08      0.14       249
        true   

### add sentiment

In [69]:
train_df['sentiment'] = train_df['sentence_processed'].apply(calculate_sentiment)
valid_df['sentiment'] = valid_df['sentence_processed'].apply(calculate_sentiment)
test_df['sentiment'] = test_df['sentence_processed'].apply(calculate_sentiment)

In [71]:
sentiment_train = train_df['sentiment'].values.reshape(-1, 1)
sentiment_test = test_df['sentiment'].values.reshape(-1, 1)

X_train = np.hstack((tfidf_train.toarray(), sentiment_train))
X_test = np.hstack((tfidf_test.toarray(), sentiment_test))

In [76]:
models_2 = train_models(X_train, y_train)


KeyboardInterrupt



In [ ]:
model_result(models_2)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer